In [19]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf


physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(
 #   physical_devices[0], True
#)
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [20]:
df = pd.read_csv("../tags_processed_stages/dafre_tags.csv")
num_classes = df['tags_cat4'].nunique()
print(num_classes)

3486


In [21]:
len(df)

463437

In [22]:

class_counts = df['tags_cat4'].value_counts()
df_filtered = df[df['tags_cat4'].isin(class_counts[class_counts >= 2].index)]
train_df, temp_df = train_test_split(df_filtered, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

common_classes = set(train_df['tags_cat4']).intersection(set(val_df['tags_cat4']), set(test_df['tags_cat4']))

# Filter the datasets to include only common classes
train_df = train_df[train_df['tags_cat4'].isin(common_classes)]
val_df = val_df[val_df['tags_cat4'].isin(common_classes)]
test_df = test_df[test_df['tags_cat4'].isin(common_classes)]
print(val_df['tags_cat4'].nunique())

3095


In [23]:
train_datagen = ImageDataGenerator(
    rescale=1./255, # Rescale the image by normalizing it.
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [24]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [25]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../fullMin256/', # Replace with the path to the directory where images are stored
    x_col='dir', # Column in dataframe that contains the filenames
    y_col='tags_cat4', # Column in dataframe that has the target labels
    target_size=(150, 150), # Size to which all images will be resized
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../fullMin256/',
    x_col='dir',
    y_col='tags_cat4',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 366503 validated image filenames belonging to 3095 classes.
Found 46064 validated image filenames belonging to 3095 classes.
Found 46028 validated image filenames belonging to 3095 classes.


In [36]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3095, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Callbacks
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6, mode='min', verbose=1)

# Training
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    verbose=1,
    steps_per_epoch=len(train_generator),  
    validation_steps=len(validation_generator), 
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/20
 3050/11454 [======>.......................] - ETA: 18:01 - loss: 7.2950 - accuracy: 0.0636

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

tra_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(tra_acc, label = "Training Accuracy")
plt.plot(val_acc, label = "Validation Accuracy")
plt.legend()
plt.title("Training vs Validation Acc")

tra_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.subplot(2,2,2)
plt.plot(tra_loss, label = "Training Loss")
plt.plot(val_loss, label = "Validation Loss")
plt.legend()
plt.title("Training Loss vs Validation Loss")
plt.show()